In [1]:
MAX_NUM_THREADS = 4

import ctypes
mkl_rt = ctypes.CDLL('libmkl_rt.so')
#print(mkl_rt.mkl_get_max_threads())
mkl_get_max_threads = mkl_rt.mkl_get_max_threads
def mkl_set_num_threads(cores):
    mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(cores)))

mkl_set_num_threads(MAX_NUM_THREADS)
print(f'Number of threads was limited to {mkl_get_max_threads()}.')

Number of threads was limited to 4.


In [2]:
import os
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import numpy_tools
import loaders.eth80 as eth80
import loaders.smni_eeg as smni_eeg

from experiment_tools import temporal_sscal
from experiment_tools import kmeans_gmm_clustering_experiment
from experiment_tools import hac_clustering_experiment


In [3]:
data_eth80_dirname = '/home/hariyuki/data/eth80/'
data_smni_dirname = '/home/hariyuki/data/eeg_smni/'

In [4]:
# load ETH80
data_eth80_path = os.path.join(data_eth80_dirname, 'eth80-cropped-close128')

image_shape = 32
Nclasses, Nobjects = 8, 10
Npixels = int(round(image_shape**2))

data, labels, classes = eth80.eth80_dataset(data_eth80_path, image_shape)
classes_reverse_dict = dict((y, x) for x, y in classes.items())
Nsamples, Nangles, _, _, Ncolors = data.shape
data = numpy_tools.reshape_np(data, [Nangles, -1, Ncolors], order='F', use_batch=True)
permutation = [0, 2, 1, 3]
data = np.transpose(data, permutation)

n = [Npixels, Nangles, Ncolors]

eth80_dataset = {
    'data': data,
    'n': n,
    'labels': labels,
    'classes': classes,
    'classes_reverse_dict': classes_reverse_dict
}

In [5]:
# load SMNI EEG
data_smni_path = os.path.join(data_smni_dirname, 'smni_eeg_processed.npz')
df = np.load(data_smni_path)
data, labels = df['data'], df['labels']

Nsubjects, Nchannels, Ntime, Nconditions = data.shape
permutation = [0, 2, 1, 3]
data = np.transpose(data, permutation)
n = [Ntime, Nchannels, Nconditions]

smni_dataset = {
    'data': data,
    'n': n,
    'labels': labels
}

In [6]:
datasets = {
    'eth80': eth80_dataset,
    'smni': smni_dataset
}
dataset_names = list(datasets.keys())

## Clustering of raw data

In [7]:
save_results_dirname = '../results/clustering/'
os.makedirs(save_results_dirname, exist_ok=True)

### Hierarchical agglomerative clustering

In [8]:
save_results_filename = 'plain_ac'
save_results_path = os.path.join(save_results_dirname, save_results_filename)
    

preprocess = {
    'smni': temporal_sscal
}

result, affinity_names, linkage, dataset_names, n_clusters = hac_clustering_experiment(
    datasets,
    save_results_path=save_results_path,
    preprocess=preprocess,
    verbose=True
)

			 eth80
		 Affinity: l1
	 Linkage: complete
ARI=0.475 AMI=0.629 FMI=0.548 
	 Linkage: average
ARI=0.470 AMI=0.672 FMI=0.556 
		 Affinity: l2
	 Linkage: complete
ARI=0.395 AMI=0.618 FMI=0.503 
	 Linkage: average
ARI=0.389 AMI=0.639 FMI=0.523 
		 Affinity: cosine
	 Linkage: complete
ARI=0.389 AMI=0.636 FMI=0.518 
	 Linkage: average
ARI=0.409 AMI=0.643 FMI=0.537 
		 Affinity: canberra
	 Linkage: complete
ARI=0.395 AMI=0.564 FMI=0.489 
	 Linkage: average
ARI=0.436 AMI=0.696 FMI=0.561 
		 Affinity: correlation
	 Linkage: complete
ARI=0.515 AMI=0.670 FMI=0.587 
	 Linkage: average
ARI=0.568 AMI=0.749 FMI=0.638 
		 Affinity: rbf
	 Linkage: complete
ARI=0.016 AMI=0.032 FMI=0.327 
	 Linkage: average
ARI=0.016 AMI=0.032 FMI=0.327 
			 smni
		 Affinity: l1
	 Linkage: complete
ARI=0.023 AMI=0.030 FMI=0.529 
	 Linkage: average
ARI=0.013 AMI=0.009 FMI=0.729 
		 Affinity: l2
	 Linkage: complete
ARI=0.161 AMI=0.104 FMI=0.601 
	 Linkage: average
ARI=0.013 AMI=0.009 FMI=0.729 
		 Affinity: cosine
	 Lin

### K-means / GMM clustering

In [9]:
save_results_filename = 'plain_kmeans_gmm'
save_results_path = os.path.join(save_results_dirname, save_results_filename)
Ntrials = 20
np_random_seed = 720

preprocess = {
    'smni': temporal_sscal
}

result, dataset_names, random_states, clust_alg_names, n_clusters = (
    kmeans_gmm_clustering_experiment(
        datasets,
        save_results_path=save_results_path,
        preprocess=preprocess,
        Ntrials=Ntrials,
        np_random_seed=np_random_seed,
        verbose=True
    )
)


			 eth80
	 kmeans min/mean/median/max
ARI=0.390/0.533/0.530/0.641
AMI=0.582/0.690/0.693/0.767
FMI=0.479/0.597/0.594/0.682
	 GMM min/mean/median/max
ARI=0.320/0.475/0.477/0.586
AMI=0.526/0.639/0.653/0.741
FMI=0.433/0.549/0.548/0.642
			 smni
	 kmeans min/mean/median/max
ARI=0.160/0.196/0.203/0.220
AMI=0.099/0.123/0.127/0.141
FMI=0.604/0.624/0.629/0.635
	 GMM min/mean/median/max
ARI=0.008/0.156/0.175/0.205
AMI=0.003/0.103/0.115/0.135
FMI=0.520/0.597/0.607/0.629
